In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install SimpleITK
!pip install pyradiomics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 49.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Discarding https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz (from https://pypi.org/simple/pyradiomics/): Requested pyradiomics from https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz has inconsistent version: expected '3.1.0', but metadata has '3.0.1a1'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 19.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11

In [3]:
import os
import SimpleITK as sitk
import pandas as pd
import numpy as np
from radiomics import featureextractor
import logging

# Initialize logging
logging.basicConfig(filename='feature_extraction.log', level=logging.INFO,
                    format='%(asctime)s - %(message)s')

# Initialize PyRadiomics extractor
extractor = featureextractor.RadiomicsFeatureExtractor()
extractor.settings['force2D'] = True  # Enable angle-wise feature extraction in 2D
extractor.settings['enableAngleAverage'] = False  # Disable angle aggregation
extractor.settings['binWidth'] = 25  # Example bin width (adjust as needed)
extractor.settings['resampledPixelSpacing'] = None  # Use original spacing
extractor.settings['interpolator'] = 'sitkBSpline'  # Interpolation method
extractor.settings['normalize'] = True  # Normalize images before feature extraction

# Paths to image and mask folders
image_folder = "/content/drive/MyDrive/FYP/Dataset/Normal"
mask_folder = "/content/drive/MyDrive/FYP/Dataset/Normal_Masks"

# Prepare the output CSV files
output_csv = "/content/drive/MyDrive/FYP/extracted_features_angle_wise_Normal.csv"
skipped_csv = "/content/drive/MyDrive/FYP/skipped_images.csv"
feature_data = []
skipped_data = []

# Counters for skipped images
single_voxel_count = 0
no_label_count = 0
dimension_issue_count = 0

# Get all image and mask files (assuming they are paired and ordered in the folders)
image_files = sorted(os.listdir(image_folder))
mask_files = sorted(os.listdir(mask_folder))

# Loop through each image and corresponding mask
for img_file, mask_file in zip(image_files, mask_files):
    image_path = os.path.join(image_folder, img_file)
    mask_path = os.path.join(mask_folder, mask_file)

    try:
        # Read the image and mask
        image = sitk.ReadImage(image_path)
        mask = sitk.ReadImage(mask_path)

        # Convert images to grayscale if necessary
        if image.GetNumberOfComponentsPerPixel() > 1:
            image = sitk.VectorIndexSelectionCast(image, 0)

        if mask.GetNumberOfComponentsPerPixel() > 1:
            mask = sitk.VectorIndexSelectionCast(mask, 0)

        # Ensure images are of type UInt8 or UInt16
        image = sitk.Cast(image, sitk.sitkUInt8)
        mask = sitk.Cast(mask, sitk.sitkUInt8)

        # Check if the image and mask sizes match
        if image.GetSize() != mask.GetSize():
            logging.warning(f"Resizing mask for {img_file} to match image dimensions.")
            mask = sitk.Resample(mask, referenceImage=image, transform=sitk.Transform(),
                                 interpolator=sitk.sitkNearestNeighbor, defaultPixelValue=0, outputPixelType=sitk.sitkUInt8)

        # Check mask dimensions
        if len(mask.GetSize()) < 2:
            logging.error(f"Mask for {img_file} has too few dimensions. Skipping.")
            dimension_issue_count += 1
            skipped_data.append({"Image": img_file, "Reason": "Dimension issue"})
            continue

        # Convert mask to NumPy array and check for valid labels
        mask_array = sitk.GetArrayFromImage(mask)
        unique_labels = np.unique(mask_array)

        if not any(label in unique_labels for label in [255, 1, 128]):
            logging.warning(f"No valid label found in mask for {img_file}. Skipping.")
            no_label_count += 1
            skipped_data.append({"Image": img_file, "Reason": "Missing valid label"})
            continue

        # Check if the mask contains only a single segmented voxel
        num_segmented_voxels = (mask_array == 255).sum()

        if num_segmented_voxels <= 1:
            logging.warning(f"Mask for {img_file} contains only {num_segmented_voxels} segmented voxel(s). Skipping.")
            single_voxel_count += 1
            skipped_data.append({"Image": img_file, "Reason": "Single voxel segmentation"})
            continue

        # Extract angle-wise features
        logging.info(f"Extracting angle-wise features for {img_file}...")
        features = extractor.execute(image, mask)

        # Store features in a dictionary with explicit angle labeling
        feature_row = {"Image": img_file}

        # Define angles and feature classes
        angles = [0, 45, 90, 135]

        # Extract features based on angle specifications
        for feature_name, feature_value in features.items():
            if "_angle" in feature_name:
                # Handle angle-specific features
                for angle in angles:
                    if f"_angle_{angle}" in feature_name:
                        feature_row[f"{feature_name}_angle_{angle}"] = feature_value

            else:
                # For non-angle features, save them directly
                feature_row[feature_name] = feature_value

        # Append the row to the data list
        feature_data.append(feature_row)

    except Exception as e:
        logging.error(f"Feature extraction failed for {img_file}: {str(e)}")
        skipped_data.append({"Image": img_file, "Reason": f"Extraction error: {str(e)}"})

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(feature_data)

# Save the extracted features to CSV
df.to_csv(output_csv, index=False)
print(f"Angle-wise feature extraction complete. Results saved to {output_csv}")

# Save skipped data to CSV
skipped_df = pd.DataFrame(skipped_data)
skipped_df.to_csv(skipped_csv, index=False)
print(f"Skipped images and reasons saved to {skipped_csv}")

# Print summary of skipped counts
print(f"Number of masks skipped due to single voxel segmentation: {single_voxel_count}")
print(f"Number of masks skipped due to missing label 255: {no_label_count}")
print(f"Number of masks skipped due to dimension issues: {dimension_issue_count}")


Streaming output truncated to the last 5000 lines.
INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Computing glcm
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
INFO:radiomics.featureextractor:Computing gldm
INFO:radiomics.featureextractor:Computing glrlm
INFO:radiomics.featureextractor:Computing glszm
INFO:radiomics.featureextractor:Computing ngtdm
INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask
Shape features are only available 3D input (for 2D input, use shape2D). Fou

Angle-wise feature extraction complete. Results saved to /content/drive/MyDrive/FYP/extracted_features_angle_wise_Normal.csv
Skipped images and reasons saved to /content/drive/MyDrive/FYP/skipped_images.csv
Number of masks skipped due to single voxel segmentation: 637
Number of masks skipped due to missing label 255: 1981
Number of masks skipped due to dimension issues: 0
